In [ ]:
%matplotlib inline
import pathlib as pl
import time

import contextily as cx
import flopy
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
cx_provider = cx.providers.USGS.USTopo

In [ ]:
def get_flow(bobj, totim):
    Q = bobj.get_data(text="SFR", totim=totim)[0]["q"].sum()
    Q += bobj.get_data(text="DRN", totim=totim)[0]["q"].sum()
    Q += bobj.get_data(text="EVTA", totim=totim)[0]["q"].sum()
    return Q

In [ ]:
ws = pl.Path("mf6_transient_base")

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=ws)

In [ ]:
gwf = sim.get_model()

In [ ]:
gwf.modelgrid.set_coord_info(xoff=529999.38, yoff=3420183.60, angrot=0, crs="EPSG:26912")
gwf.modelgrid

In [ ]:
extent = gwf.modelgrid.extent

Add well file for perturbation

In [ ]:
ca_wel = flopy.mf6.ModflowGwfwel(gwf, maxbound=1, pname="ca_well")

In [ ]:
gwf.get_package_list()

Change workspace and write files

In [ ]:
new_ws = pl.Path("capture_analysis")
sim.set_sim_path(new_ws)
sim.write_simulation()

In [ ]:
sim.run_simulation()

In [ ]:
hobj = gwf.output.head()

In [ ]:
times = hobj.get_times()
times[0], times[-1]

In [ ]:
hds = hobj.get_data(totim=times[-1])

In [ ]:
cobj = gwf.output.budget()

In [ ]:
cobj.get_unique_record_names()

In [ ]:
spdis = cobj.get_data(text="DATA-SPDIS", totim=times[-1])[0]
spdis
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf, head=hds)

In [ ]:
mosaic = [
    ["a", "b"], ["a", "b"], ["a", "b"], ["a", "b"], ["a", "b"],
    ["a", "b"], ["a", "b"], ["a", "b"], ["a", "b"], ["a", "b"],
    ["c", "c"],
]

In [ ]:
plt_ratio = float(gwf.modelgrid.nrow) / float(gwf.modelgrid.ncol)
plt_ratio

In [ ]:
vmin, vmax = 1120, 1680
levels = [1200, 1300, 1400, 1500, 1600]
width = 7.5
height = (plt_ratio + 0.1) * (width / 2)
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplot_mosaic(mosaic, sharey=True, figsize=(width, height), layout="constrained")
          
    ax = axs["a"]
    ax.set_xlim(gwf.modelgrid.extent[0:2])
    ax.set_ylim(gwf.modelgrid.extent[2:])
    ax.yaxis.offsetText.set_visible(False)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=3, extent=gwf.modelgrid.extent)
    cx.add_basemap(ax, crs=gwf.modelgrid.crs, attribution=False, source=cx_provider)
    mm.plot_array(hds, vmin=vmin, vmax=vmax)
    mm.plot_bc("SFR", color="cyan")
    mm.plot_vector(qx, qy, istep=5, jstep=5, normalize=True)
    cs = mm.contour_array(hds, levels=levels, colors="black", linewidths=0.5)
    ax.clabel(cs, cs.levels, inline=True, fmt="%1.0f", fontsize=6, inline_spacing=1)
    flopy.plot.styles.heading(ax=ax, idx=0, heading="Transient Layer 4")
    flopy.plot.styles.xlabel(ax=ax, label="x-coordinate, m")
    flopy.plot.styles.ylabel(ax=ax, label="y-coordinate (x 10$^6$), m")

    ax = axs["b"]
    ax.set_xlim(gwf.modelgrid.extent[0:2])
    ax.set_ylim(gwf.modelgrid.extent[2:])
    ax.yaxis.offsetText.set_visible(False)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=4)
    cx.add_basemap(ax, crs=gwf.modelgrid.crs, attribution=False, source=cx_provider)
    v = mm.plot_array(hds, vmin=vmin, vmax=vmax)
    mm.plot_bc("SFR", color="cyan")
    mm.plot_vector(qx, qy, istep=5, jstep=5, normalize=True)
    cs = mm.contour_array(hds, levels=levels, colors="black", linewidths=0.5)
    ax.clabel(cs, cs.levels, inline=True, fmt="%1.0f", fontsize=6, inline_spacing=1)
    flopy.plot.styles.heading(ax=ax, idx=1, heading="Transient Layer 5")
    flopy.plot.styles.xlabel(ax=ax, label="x-coordinate, m")

    ax = axs["c"]
    ax.axis("off")
    cbar = plt.colorbar(v, ax=ax, orientation="horizontal", extend="both", ticks=levels, fraction=0.5)
    cbar.ax.set_xlabel("Simulated head, m", rotation=0)
    fig.savefig("results_trans.png", dpi=300)

### Get base stream baseflow

In [ ]:
Q_base = get_flow(cobj, times[-1])
Q_base

In [ ]:
nlay, nrow, ncol = gwf.dis.nlay.array, gwf.dis.nrow.array, gwf.dis.ncol.array
nlay, nrow, ncol

In [ ]:
idomain = gwf.dis.idomain.array
idomain.shape

In [ ]:
cf = np.full((nlay, nrow, ncol), -1.0, dtype=float)

In [ ]:
k = 3
dq = -10.0

In [ ]:
sample = 2 # calculate cf for every sample location
sample_arr = np.array([sample])
ilocs = []
for i in range(0, nrow, sample):
    for j in range(0, ncol, sample):
        if idomain[k, i, j] > 0:
            ilocs.append((i, j))
ilocs = np.array(ilocs)

In [ ]:
total_cells = ilocs.shape[0]
total_cells

In [ ]:
cf_path = "capture_fraction_transient.npz"

In [ ]:
t0 = time.perf_counter()
for idx, (i,j) in enumerate(ilocs):
   
    gwf.ca_well.stress_period_data = {0: [(k, i, j, dq)]}
    gwf.ca_well.write()
    
    sim.run_simulation(silent=True)
    Q = get_flow(gwf.output.budget(), times[-1])
    
    fraction = abs(Q - Q_base) / abs(dq)
    cf[k, i, j] = fraction

    progress = (idx + 1) / total_cells    
    message = (
        f"Location: ({idx:5d}) ({k}, {i:3d}, {j:3d}) " + 
        f"Capture fraction: {fraction: 5.3f} " + 
        f"Percent complete: {progress:7.3%} " + 
        f"Run time: {(time.perf_counter() - t0)/60:9.4f} minutes"
    )
    print(message, end="\r")

    if idx % 10 == 0:
        np.savez_compressed(cf_path, capture_fraction=cf, sample_size=sample_arr)

t1 = time.perf_counter()
print(f"\n\nSimulation time: {(t1 - t0)/3600. :9.4f} hours")

In [ ]:
np.savez_compressed(cf_path, capture_fraction=cf, sample_size=sample_arr)